In [13]:
import os
import polars as pl

In [73]:
base = "/mnt/d/data/"
com = "Com20"
com_dirs = [os.path.join(base, d) for d in os.listdir(base) if com in d and os.path.isdir(os.path.join(base, d))]
com_dirs

['/mnt/d/data/Com20_repl',
 '/mnt/d/data/Com20_single',
 '/mnt/d/data/Com20_transwell_interact',
 '/mnt/d/data/Com20_val']

In [87]:
comb = pl.read_excel(os.path.join(com_dirs[1], "20230327_WKL_P0024.xlsx"), read_options={"has_header": False})
comb

column_1,column_2,column_3
str,str,str
"""P1-A1""","""Basic_FIA_Iso_…","""NT5001_B1_P1-A…"
"""P1-A1""","""Basic_FIA_Iso_…","""NT5001_B1_P1-A…"
"""P1-A2""","""Basic_FIA_Iso_…","""NT5002_B1_P1-A…"
"""P1-A2""","""Basic_FIA_Iso_…","""NT5002_B1_P1-A…"
"""P1-A3""","""Basic_FIA_Iso_…","""NT5003_B1_P1-A…"
…,…,…
"""P2-B1""","""Basic_FIA_Iso_…","""TH_B2_P2-B1_ne…"
"""P2-B2""","""Basic_FIA_Iso_…","""mGAM_B3_P2-B2_…"
"""P2-B2""","""Basic_FIA_Iso_…","""mGAM_B3_P2-B2_…"


In [88]:
orgs = pl.read_csv(os.path.join(base, f"{com}_microbes.tsv"), separator="\t")
orgs

ID,Organism
str,str
"""NT5001""","""Phocaeicola vu…"
"""NT5002""","""Bacteroides un…"
"""NT5003""","""Bacteroides fr…"
"""NT5004""","""Bacteroides th…"
"""NT5006""","""Thomasclavelia…"
…,…
"""NT5048""","""Bariatricus co…"
"""NT5071""","""Parabacteroide…"
"""NT5072""","""Streptococcus …"


In [103]:
rows = []
for row in comb[::2].iter_rows():
    row = row[2]
    rows.append(pl.DataFrame([1 if i in row else 0 for i in orgs["ID"]]).transpose())
comb_one_hot_df = pl.concat(rows)
comb_one_hot_df.columns = [str(i) for i in range(len(orgs))]
comb_one_hot_df[20:30]

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [104]:
comb_one_hot_df.write_csv(os.path.join(com_dirs[1], "comb_one_hot.tsv"), separator="\t")